In [4]:
from src.data.make_dataset import * 
from src.data.hex_utils import * 

In [5]:
crs = "epsg:2333"
lonlimits = [116.1, 116.7]
latlimits = [39.7, 40.1]
n_rows = 50

limits = gpd.points_from_xy(x=lonlimits, y=latlimits, crs="epsg:4326")
limits = limits.to_crs(crs)

In [24]:
df = read_all_users("../data/raw/Geolife Trajectories 1.3/Data")
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat, crs="epsg:4326")
)

# project .to_crs("epsg:2333")
gdf = gdf.to_crs(crs)
gdf = gdf.cx[limits[0].x : limits[1].x, limits[0].y : limits[1].y]

gdf = gdf.assign(t_idx=gdf.groupby(["user", "trajectory"]).ngroup() + 1)

Processing user folders:   3%|▎         | 6/182 [00:03<01:52,  1.57it/s]


KeyboardInterrupt: 

In [ ]:
gdf = gdf.reset_index(drop=True)
xy = gdf['geometry'].get_coordinates()
gdf = pd.concat([gdf, xy], axis=1)
gdf = gdf.drop(columns=["geometry", 'lon', 'lat'])
gdf['timediff'] = gdf.groupby("t_idx")["datetime"].transform(lambda x: x - x.shift(1))

In [27]:
a = np.ones((10, 2)) * 2
b = np.zeros((10, 2))

np.linalg.norm(a-b, axis=-1)

array([2.82842712, 2.82842712, 2.82842712, 2.82842712, 2.82842712,
       2.82842712, 2.82842712, 2.82842712, 2.82842712, 2.82842712])

In [55]:
tmp = []
for t_idx, tdf in gdf.groupby("t_idx"):
    xy = tdf[['x', 'y']]
    tdf["dist"] = np.linalg.norm(xy - xy.shift(1), axis=-1)
    tmp.append(tdf)
gdf = pd.concat(tmp)
gdf

,lat,lon,datetime,trajectory,user,t_idx,timediff,x,y,dist
312042,39.984702,116.318417,2008-10-23 02:53:04,0,0,1,NaT,1.995438e+07,4.441405e+06,NaN
312043,39.984683,116.318450,2008-10-23 02:53:10,0,0,1,0 days 00:00:06,1.995438e+07,4.441403e+06,3.529644
312044,39.984686,116.318417,2008-10-23 02:53:15,0,0,1,0 days 00:00:05,1.995438e+07,4.441403e+06,2.845456
312045,39.984688,116.318385,2008-10-23 02:53:20,0,0,1,0 days 00:00:05,1.995437e+07,4.441403e+06,2.749191
312046,39.984655,116.318263,2008-10-23 02:53:25,0,0,1,0 days 00:00:05,1.995436e+07,4.441399e+06,11.073900
...,...,...,...,...,...,...,...,...,...,...
12420308,39.989650,116.300717,2007-12-07 13:52:57,14,181,17119,0 days 00:01:07,1.995283e+07,4.441864e+06,259.596741
12420309,39.989817,116.299167,2007-12-07 13:53:37,14,181,17119,0 days 00:00:40,1.995270e+07,4.441875e+06,134.004899
12420310,39.989683,116.298350,2007-12-07 13:55:25,14,181,17119,0 days 00:01:48,1.995263e+07,4.441856e+06,71.482504
12420311,39.990017,116.297883,2007-12-07 13:56:31,14,181,17119,0 days 00:01:06,1.995259e+07,4.441891e+06,54.528176


In [58]:
gdf["dist"] = gdf["dist"].fillna(0)

# Calculate speed
gdf["speed"] = gdf["dist"] / gdf["timediff"].dt.total_seconds()

# Filter out all trajectories where the average speed is above 30 m/s $\approx$ chinese driving speed limit
speed_mask = gdf.groupby(["t_idx"])["speed"].mean() <= 30
valid_traj_idx = speed_mask[speed_mask].index
gdf = gdf[gdf["t_idx"].isin(valid_traj_idx)]

In [59]:
gdf.to_pickle("../data/processed/geolife.pkl")

In [47]:
import pickle
with open("../data/processed/geolife.pkl", 'rb') as f:
    gdf = pickle.load(f)

In [48]:
df = gdf
n_rows=50

In [49]:
lonlat_offset = np.array([limits[0].x, limits[0].y])
hex_size = get_hex_size(limits=limits, n_rows=n_rows)

xy = df[['x', 'y']].values
qr = coords_to_hex(xy, hex_size, lonlat_offset)
hdf = pd.DataFrame(qr, columns=["q", "r"])
hdf["datetime"] = df["datetime"]
hdf["t_idx"] = df["t_idx"]
hdf['user'] = df['user']

In [41]:
def interpolate_cell_jumps(df):
    qr1 = df[["q", "r"]].shift(1).values
    qr2 = df[["q", "r"]].values

    df["cell_dist"] = cell_distance(qr1, qr2)
    df["cell_dist"] = df["cell_dist"].fillna(1)
    if not (df["cell_dist"] > 1).any():
        return df[df["cell_dist"] >= 1]
    else:
        t1 = df[["datetime"]].shift(1).values
        t2 = df[["datetime"]].values

        qri = []
        ti = []

        for i, ni in enumerate(df["cell_dist"]):
            if not np.isnan(ni) and ni > 1:
                t = np.arange(1, ni) / ni
                ti.append(t1[i] + (t2[i] - t1[i]) * t[:, None])
                qri.append(qr1[i] + (qr2[i] - qr1[i]) * t[:, None])
        new_rows = pd.DataFrame(cell_round(np.concatenate(qri)), columns=["q", "r"])
        new_rows["datetime"] = np.concatenate(ti)
        new_rows["t_idx"] = df.iloc[0]["t_idx"]
        return pd.concat([df[df["cell_dist"] >= 1], new_rows]).sort_values("datetime")

In [50]:
tmp = []
for t_idx, dft in tqdm(hdf.groupby('t_idx')):
    tmp.append(interpolate_cell_jumps(dft))

hdf = pd.concat(tmp)

100%|██████████| 15913/15913 [02:08<00:00, 123.74it/s]


In [51]:
hdf = hdf.groupby("t_idx").filter(lambda x: len(x) >= 4)

In [52]:
hdf.to_pickle(f"../data/processed/geolife_hex_{n_rows}.pkl")

In [53]:
hdf

,q,r,datetime,t_idx,user,cell_dist
312042,23,17,2008-10-23 02:53:04,1.0,0.0,1.0
312091,23,16,2008-10-23 02:57:10,1.0,0.0,1.0
312093,23,17,2008-10-23 02:57:20,1.0,0.0,1.0
312095,23,16,2008-10-23 02:57:30,1.0,0.0,1.0
312096,23,17,2008-10-23 02:57:35,1.0,0.0,1.0
...,...,...,...,...,...,...
12419898,28,21,2007-12-31 06:01:01,17110.0,181.0,1.0
12419902,27,22,2007-12-31 06:04:27,17110.0,181.0,1.0
12419917,26,22,2007-12-31 07:13:18,17110.0,181.0,1.0
12419925,25,23,2007-12-31 07:17:59,17110.0,181.0,1.0


In [61]:
pd.get_dummies(hdf, columns=['user'])

,q,r,datetime,t_idx,cell_dist,user_0.0,user_1.0,user_2.0,user_3.0,user_4.0,...,user_168.0,user_169.0,user_170.0,user_173.0,user_174.0,user_176.0,user_177.0,user_178.0,user_179.0,user_181.0
312042,23,17,2008-10-23 02:53:04,1.0,1.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
312091,23,16,2008-10-23 02:57:10,1.0,1.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
312093,23,17,2008-10-23 02:57:20,1.0,1.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
312095,23,16,2008-10-23 02:57:30,1.0,1.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
312096,23,17,2008-10-23 02:57:35,1.0,1.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12419898,28,21,2007-12-31 06:01:01,17110.0,1.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
12419902,27,22,2007-12-31 06:04:27,17110.0,1.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
12419917,26,22,2007-12-31 07:13:18,17110.0,1.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
12419925,25,23,2007-12-31 07:17:59,17110.0,1.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [60]:
grid, _ = make_hexgrid(limits, hex_size)
axial = hdf[hdf.t_idx == 100][['q', 'r']].values


KEYS = {"axial": ["q", "r"], "oddq": ["col", "row"]}
if axial is not None:
        keys = KEYS["axial"]
        idx = axial
else:
    keys = KEYS["oddq"]

map = folium.Map(location=(40.2, 116.383331), zoom_start=8.2)
# grid = grid.set_index(keys)
pgrid = grid.to_crs("epsg:4326")
pgrid = pgrid.set_index(keys)
pgrid["visited"] = 0
pgrid.loc[idx.tolist(), "visited"] = 1
pgrid = pgrid.reset_index(names=keys)

grid_json = pgrid.to_json()
tooltip = folium.GeoJsonTooltip(fields=keys)

def get_style(feature):
    props = feature["properties"]
    if [props[key] for key in keys] in idx.tolist():
        return STYLE_VISITED
    else:
        return STYLE_GRID

grid_layer = folium.GeoJson(
    grid_json,
    name="non-visited",
    style_function=get_style,
    tooltip=tooltip,
)
grid_layer.add_to(map)
map
